In [129]:
import pprint, requests, json

### List all datasets

In [130]:
def get_all_datasets():
    url = "https://opendata.urbanplatform.portodigital.pt/api/3/action/package_list";
    r = requests.get(url)
    return r.json()["result"]

In [132]:
datasets = get_all_datasets()
# print(datasets)

### Get Information for custom dataset, by name
If `url_only` is true, returns location of data, else a tuple of `(dataset-resource, FORMAT, URL)`

In [123]:
def get_dataset_info(dataset, url_only=True):
    # returns a generator for (dataset-resource, FORMAT, URL) or just the URL
    url = "https://opendata.urbanplatform.portodigital.pt/api/3/action/package_show?id=%s" % dataset
    j = requests.get(url).json()
    if not j["success"]: print(j["error"]); return
    for resource in j['result']['resources']:
        if url_only: yield resource["url"]
        else: yield (resource["name"], resource["format"], resource["url"])

In [139]:
alojamento = list(get_dataset_info(datasets[1]))[0]
print(alojamento)
print(list(get_dataset_info(datasets[1], False)))

https://servsig.cm-porto.pt/arcgis/rest/services/OpenData_APD/OpenData_APD/MapServer/35
[('Alojamento Local', 'Esri REST', 'https://servsig.cm-porto.pt/arcgis/rest/services/OpenData_APD/OpenData_APD/MapServer/35')]


In [127]:
# exemplo com vários
list(get_dataset_info("porto-meteorologia"))

['https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?type=WeatherObserved',
 'https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?type=WeatherForecast']

In [161]:
# exemplo com csv que dá "Access denied"
list(get_dataset_info("apdg-zonas-de-estacionamento-pago"))

{'message': 'Access denied: User  not authorized to read package d5d1106b-5db6-4434-b1b2-9083d9fb1f49', '__type': 'Authorization Error'}


[]

### Get data from url
 * Set the query params
 * Specify how to parse result
 * function to handle request result and return generator of datapoints

In [162]:
DEFAULT_REQ_PARAMS = {'where': "1=1", 'returnGeometry': 'true','orderByFields': 'objectid ASC', 'outSR': '4326'}

In [163]:
def remove_useless_from_dict(dic):
    return {k: v for k, v in dic.items() if v and v!=" "}
def parse_features_geojson(x):
    features = x["properties"]
    features["coordinates"] = x["geometry"]["coordinates"]
    return remove_useless_from_dict(features)
def parse_features_json(x): return remove_useless_from_dict(x["attributes"])

In [165]:
def get_dataset_data(dataset_url, req_params={"f":"json"}, f="geojson", fields="*"):
    # default format could be json, but this gives x, y and not lat, lon
    params = DEFAULT_REQ_PARAMS; params.update(req_params); params["f"] = f; params["outFields"]=fields
    data = requests.get(dataset_url + "/query", params=params).json()
    get_attributes = parse_features_geojson if f=="geojson" else parse_features_json
    return map(parse_features_geojson, data["features"])

In [168]:
alojamentos = list(get_dataset_data(alojamento))
print("There are %d alojamentos" % len(alojamentos))
print(alojamentos)

There are 1000 alojamentos
[{'objectid': 1, 'data_levan': 1488758400000, 'cod_dmpu': '140747', 'cod_topo': 'RCAMO0', 'n_pol': '77', 'nome_aloj': 'Metro Apartment', 'ano_reg': 2016, 'ano_ab': 2016, 'n_reg': '35375/AL', 'qual_tit': 'Proprietario', 'tit_tipo': 'Pessoa singular (empresário em nome individual)', 'pos_1951': 'S', 'morada': 'Rua Rua Camões, 77 1º 1.1', 'modalidade': 'Apartamento', 'cod_postal': '4000-144', 'data_reg': 1475193600000, 'data_ab': 1477958400000, 'coordinates': [-8.610274113044303, 41.1525537929996]}, {'objectid': 2, 'data_levan': 1488758400000, 'cod_dmpu': '140749', 'cod_topo': 'RCAMO0', 'n_pol': '327', 'nome_aloj': 'Be Happy Flat', 'ano_reg': 2016, 'ano_ab': 2016, 'n_reg': '23120/AL', 'qual_tit': 'Proprietario', 'tit_tipo': 'Pessoa singular (empresário em nome individual)', 'pos_1951': 'S', 'morada': 'Rua Rua Camões, 327 1.º Hab. 1.5', 'modalidade': 'Apartamento', 'cod_postal': '4000-145', 'data_reg': 1451952000000, 'data_ab': 1451952000000, 'coordinates': [-8.6